# Importações

In [1]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras import Input
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

# Carregando e Compreendendo os Dados

In [3]:
df = pd.read_csv('dataset.csv')
df.head()

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [4]:
df.shape

(9841, 51)

In [5]:
df.FLAG.value_counts()

FLAG
0    7662
1    2179
Name: count, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 51 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Unnamed: 0                                            9841 non-null   int64  
 1   Index                                                 9841 non-null   int64  
 2   Address                                               9841 non-null   object 
 3   FLAG                                                  9841 non-null   int64  
 4   Avg min between sent tnx                              9841 non-null   float64
 5   Avg min between received tnx                          9841 non-null   float64
 6   Time Diff between first and last (Mins)               9841 non-null   float64
 7   Sent tnx                                              9841 non-null   int64  
 8   Received Tnx                                          9841

# Limpeza dos Dados

In [7]:
df.columns

Index(['Unnamed: 0', 'Index', 'Address', 'FLAG', 'Avg min between sent tnx',
       'Avg min between received tnx',
       'Time Diff between first and last (Mins)', 'Sent tnx', 'Received Tnx',
       'Number of Created Contracts', 'Unique Received From Addresses',
       'Unique Sent To Addresses', 'min value received', 'max value received ',
       'avg val received', 'min val sent', 'max val sent', 'avg val sent',
       'min value sent to contract', 'max val sent to contract',
       'avg value sent to contract',
       'total transactions (including tnx to create contract',
       'total Ether sent', 'total ether received',
       'total ether sent contracts', 'total ether balance',
       ' Total ERC20 tnxs', ' ERC20 total Ether received',
       ' ERC20 total ether sent', ' ERC20 total Ether sent contract',
       ' ERC20 uniq sent addr', ' ERC20 uniq rec addr',
       ' ERC20 uniq sent addr.1', ' ERC20 uniq rec contract addr',
       ' ERC20 avg time between sent tnx', ' ERC20 

In [8]:
df.columns = [x.lower() for x in df.columns]

In [9]:
cols_to_drop = [' erc20 most sent token type',
                ' erc20_most_rec_token_type',
                'address',
                'index',
                'unnamed: 0']

In [10]:
atributos = [x for x in df.columns if (x != 'flag' and x not in cols_to_drop)]

In [11]:
atributos

['avg min between sent tnx',
 'avg min between received tnx',
 'time diff between first and last (mins)',
 'sent tnx',
 'received tnx',
 'number of created contracts',
 'unique received from addresses',
 'unique sent to addresses',
 'min value received',
 'max value received ',
 'avg val received',
 'min val sent',
 'max val sent',
 'avg val sent',
 'min value sent to contract',
 'max val sent to contract',
 'avg value sent to contract',
 'total transactions (including tnx to create contract',
 'total ether sent',
 'total ether received',
 'total ether sent contracts',
 'total ether balance',
 ' total erc20 tnxs',
 ' erc20 total ether received',
 ' erc20 total ether sent',
 ' erc20 total ether sent contract',
 ' erc20 uniq sent addr',
 ' erc20 uniq rec addr',
 ' erc20 uniq sent addr.1',
 ' erc20 uniq rec contract addr',
 ' erc20 avg time between sent tnx',
 ' erc20 avg time between rec tnx',
 ' erc20 avg time between rec 2 tnx',
 ' erc20 avg time between contract tnx',
 ' erc20 min val

In [12]:
valores_unicos = df.nunique()
valores_unicos

unnamed: 0                                              9841
index                                                   4729
address                                                 9816
flag                                                       2
avg min between sent tnx                                5013
avg min between received tnx                            6223
time diff between first and last (mins)                 7810
sent tnx                                                 641
received tnx                                             727
number of created contracts                               20
unique received from addresses                           256
unique sent to addresses                                 258
min value received                                      4589
max value received                                      6302
avg val received                                        6767
min val sent                                            4719
max val sent            

In [13]:
atributos = [x for x in atributos if x in valores_unicos.loc[(valores_unicos > 1)]]

In [14]:
df[atributos].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 38 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   avg min between sent tnx                              9841 non-null   float64
 1   avg min between received tnx                          9841 non-null   float64
 2   time diff between first and last (mins)               9841 non-null   float64
 3   sent tnx                                              9841 non-null   int64  
 4   received tnx                                          9841 non-null   int64  
 5   number of created contracts                           9841 non-null   int64  
 6   unique received from addresses                        9841 non-null   int64  
 7   unique sent to addresses                              9841 non-null   int64  
 8   min value received                                    9841

# Criação do Pipeline de Pré-Processamento

In [15]:
class PipeSteps(BaseEstimator, TransformerMixin):
    def __init__(self, columns=[]):
        self.columns = columns

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        X = X.copy()
        return X

In [16]:
class SelecionaColunas(PipeSteps):
    def transform(self, X):
        X = X.copy()
        return X[self.columns]

In [17]:
class PreenchendoDados(PipeSteps):
    def fit(self, X, y = None):
        self.means = {col: X[col].mean() for col in self.columns}
        return self

    def transform(self, X):
        X = X.copy()

        for col in self.columns:
            X[col] = X[col].fillna(self.means[col])

        return X

In [18]:
class PadronizaDados(PipeSteps):
    def fit(self, X, y = None):
        self.scaler = StandardScaler()
        self.scaler.fit(X[self.columns])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.columns] = self.scaler.transform(X[self.columns])
        return X

In [19]:
class ObtemDados(PipeSteps):
    def transform(self, X):
        X = X.copy()
        return X.values

In [20]:
pipe_preprocessamento = Pipeline([('feature_selection', SelecionaColunas(atributos)),
                                  ('fill_missing', PreenchendoDados(atributos)),
                                  ('standard_scaling', PadronizaDados(atributos)),
                                  ('returnValues', ObtemDados())])

In [21]:
X = df[atributos]

In [22]:
y = df['flag']

In [23]:
y = to_categorical(y)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
x_treino = pipe_preprocessamento.fit_transform(x_train)
x_teste = pipe_preprocessamento.fit_transform(x_test)

In [26]:
x_treino

array([[-0.23259911, -0.35751318, -0.67960935, ..., -0.01331307,
        -0.21172682, -0.2313969 ],
       [-0.23259911,  0.0138933 , -0.06181513, ..., -0.01331307,
        -0.21172682, -0.17365057],
       [-0.23259911, -0.34333968, -0.67254633, ..., -0.01331307,
        -0.21172682, -0.2313969 ],
       ...,
       [-0.23259911, -0.35751318, -0.67960389, ..., -0.01331307,
        -0.21172682, -0.28914324],
       [-0.22473065, -0.35751318, -0.67858957, ..., -0.01331307,
        -0.21172682, -0.28914324],
       [-0.23250192, -0.35750828, -0.67958981, ..., -0.01331307,
        -0.21172682, -0.28914324]])

In [27]:
x_teste

array([[-0.17363074, -0.2389954 ,  0.35405294, ..., -0.04690525,
        -0.22359372,  0.04877518],
       [-0.1218577 , -0.29065065, -0.63739158, ..., -0.04690525,
        -0.22359372, -0.27878033],
       [-0.23932993, -0.3248651 , -0.66701725, ..., -0.04690525,
        -0.22359372, -0.36066921],
       ...,
       [-0.24621445, -0.32484836, -0.66797958, ..., -0.04690525,
        -0.22359372, -0.36066921],
       [-0.24629279, -0.3246091 , -0.66391201, ..., -0.04690525,
        -0.22359372, -0.27878033],
       [-0.2452496 , -0.3248651 , -0.66777809, ..., -0.04690525,
        -0.22359372, -0.36066921]])

# Construção do Modelo de Deep Learning

In [28]:
modelo = Sequential()

In [29]:
modelo.add(Input(shape = (len(atributos),)))
modelo.add(Dense(len(atributos), activation='relu'))
modelo.add(Dense(20, activation='relu'))
modelo.add(Dense(5, activation='relu'))
modelo.add(Dense(2, activation='softmax'))

I0000 00:00:1726485593.970789    6117 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726485594.140090    6117 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726485594.140325    6117 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726485594.142454    6117 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [33]:
modelo.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [34]:
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 38)             │         1,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           105 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,379 (9.29 KB)

 Trainable params: 2,379 (9.29 KB)

 Non-trainable params: 0 (0.00 B)

# Treinamento e Avaliação do Modelo

In [35]:
%%time
modelo.fit(x_treino, y_train, validation_data=(x_teste, y_test), epochs=10)

Epoch 1/10


I0000 00:00:1726485628.932837    6420 service.cc:146] XLA service 0x716368006660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726485628.932862    6420 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6


171/216 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - accuracy: 0.7615 - loss: 0.5296

I0000 00:00:1726485630.491236    6420 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7685 - loss: 0.5153 - val_accuracy: 0.8527 - val_loss: 0.3827
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8621 - loss: 0.3247 - val_accuracy: 0.8639 - val_loss: 0.2990
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8718 - loss: 0.2670 - val_accuracy: 0.8764 - val_loss: 0.2633
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9136 - loss: 0.2421 - val_accuracy: 0.8798 - val_loss: 0.2729
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9165 - loss: 0.2167 - val_accuracy: 0.8801 - val_loss: 0.2827
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9380 - loss: 0.1866 - val_accuracy: 0.9357 - val_loss: 0.2613
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9532 - loss: 0.1590 - val_accuracy: 0.9397 - val_loss: 0.2788
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9493 - loss: 0.1517 - val_accuracy: 0.9407 - val

In [36]:
previsoes_teste = [np.argmax(x) for x in modelo.predict(x_teste)]

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [37]:
acc = metrics.accuracy_score(previsoes_teste, [np.argmax(x) for x in y_test])

In [38]:
print(f'Acurácia nos Dados de Teste: {acc:.2%}')

Acurácia nos Dados de Teste: 94.51%


In [41]:
auc = metrics.roc_auc_score([np.argmax(x) for x in y_test], modelo.predict(x_teste)[:,1])

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step


In [42]:
print(f'AUC nos Dados de Teste: {auc:,.2f}')

AUC nos Dados de Teste: 0.97


# Deploy do Modelo e Detecção de Fraudes em Novas Transações com Criptomoedas

In [44]:
novos_dados = pd.read_csv('novos_dados.csv')

In [45]:
novos_dados

,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,...,erc20 uniq sent addr.1,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name
0,2570.59,3336.01,30572.7,8,3,0,2,4,0.1,40.0,...,0.0,1.0,600.0,600.0,600.0,0.0,0.0,0.0,0.0,1.0


In [46]:
novos_dados_transformados = pipe_preprocessamento.transform(novos_dados)

In [47]:
novos_dados_transformados

array([[-0.13252061, -0.18865691, -0.57264058, -0.12789237, -0.16265097,
        -0.02199271, -0.08231218, -0.0952209 , -0.15677407, -0.05655525,
        -0.15983239, -0.03152224, -0.03659254, -0.10903682, -0.02469659,
        -0.02469659, -0.02469659, -0.19266588, -0.02319948, -0.07191711,
        -0.02469659,  0.01702814, -0.08288235, -0.02069519, -0.05078136,
        -0.02014615, -0.06532301, -0.07564193, -0.05823323, -0.27671472,
         0.01012491, -0.02056233, -0.02284285, -0.04304813, -0.03587653,
        -0.04690525, -0.22359372, -0.27878033]])

In [48]:
previsao = [np.argmax(x) for x in modelo.predict(novos_dados_transformados)]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


In [49]:
type(previsao)

list

In [52]:
print('Não é uma fraude') if previsao[0] == 0 else print('É uma fraude. Acione a verificação humana')

Não é uma fraude
